In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import torch.nn.functional as F
from torch.utils.data import Subset


import torchvision
from torchvision.models import resnet18, ResNet18_Weights, vgg16, VGG16_Weights
from torchvision import datasets
from torchvision import transforms

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image

import os
from torchsummary import summary



# from google.colab import drive
# drive.mount('/content/drive')
# #Code folder path
# %cd /content/drive/My Drive/ECE591_DL_CL_PROJECT/


# Corrupted CIFAR10

In [20]:

corruptions = ["brightness", "contrast", "defocus_blur", "elastic_transform", 
               "fog", "frost", "gaussian_blur", "gaussian_noise","glass_blur",
               "impulse_noise", "jpeg_compression", "labels", "motion_blur",
               "pixelate", "saturate", "shot_noise", "snow", "spatter", "speckle_noise","zoom_blur"]

#https://github.com/tanimutomo/cifar10-c-eval/blob/master
class CIFAR10C(datasets.VisionDataset):
    def __init__(self, root :str, name :str,transform=None, target_transform=None):
        
        assert name in corruptions
        super(CIFAR10C, self).__init__(root, transform=transform,target_transform=target_transform)
        data_path = os.path.join(root, name + '.npy')
        target_path = os.path.join(root, 'labels.npy')
        
        self.data = np.load(data_path)
        self.targets = np.load(target_path)
        
    def __getitem__(self, index):
        img, targets = self.data[index], self.targets[index]
        img = Image.fromarray(img)
        
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            targets = self.target_transform(targets)
            
        return img, targets
    
    def __len__(self):
        return len(self.data)


def extract_subset(dataset, num_subset :int, random_subset :bool):
    if random_subset:
        random.seed(0)
        indices = random.sample(list(range(len(dataset))), num_subset)
    else:
        indices = [i for i in range(num_subset)]
    return Subset(dataset, indices)

In [6]:
name="gaussian_noise"
root="./Datasets/CIFAR-10/CIFAR-10-C/"
data_path = os.path.join(root, name + '.npy')
target_path = os.path.join(root, 'labels.npy')


In [7]:
data=np.load(data_path)
targets= np.load(target_path)

In [11]:
targets


array([3, 8, 8, ..., 5, 1, 7], dtype=uint8)